# Data Exploration - Setup - Splitting

## Imports

In [33]:
# Data Handling
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Visualization
import matplotlib.pyplot as plt

# Paths
from src.__00__paths import curated_data_dir, raw_data_dir, processed_data_dir
import shutil
from pathlib import Path

# Datasets Source
import kagglehub

## Download Walmart Sales Forecast Data (Kaggle)

In [34]:
# Data files to check
data_items = [
    raw_data_dir / "features.csv",
    raw_data_dir / "stores.csv",
    raw_data_dir / "test.csv",
    raw_data_dir / "train.csv",
]

# Check and download
if all(item.exists() for item in data_items):
    print("Dataset are already downloaded.")
else:
    print("Downloading data...")
    dataset_path = Path(kagglehub.dataset_download("aslanahmedov/walmart-sales-forecast"))

    if not dataset_path.exists():
        raise FileNotFoundError("Dataset not found.")

    # Copy all files from KaggleHub Cache to `data/raw`
    for item in dataset_path.iterdir():
        target = raw_data_dir / item.name
        shutil.copy2(item, target)
    print("Dataset successfully downloaded.")

Dataset are already downloaded.


## Load Data

In [35]:
features_df = pd.read_csv(raw_data_dir / "features.csv")
features_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


> CPI, temperature, unemployment rate and fuel price have no pattern on weekly sales.

In [36]:
stores_df = pd.read_csv(raw_data_dir / "stores.csv")
stores_df.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


> Some departments has higher sales, on average others can be best. It shows us, some departments has effect on sales on some seasons like Thanksgiving.

> Stores has 3 types as A, B and C according to their sizes. Almost half of the stores are bigger than 150000 and categorized as A. According to type, sales of the stores are changing.


In [37]:
raw_train_df = pd.read_csv(raw_data_dir / "train.csv")
raw_train_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


> As expected, holiday average sales are higher than normal dates.

> Top 4 sales belongs to Christmas, Thankgiving and Black Friday times. Interestingly, 22th week of the year is the 5th best sales. It is end of May and the time when schools are closed.

> Christmas holiday introduces as the last days of the year. But people generally shop at 51th week. So, when we look at the total sales of holidays, Thankgiving has higher sales between them which was assigned by Walmart. But, when we look at the data we can understand it is not a good idea to assign Christmas sales in data to last days of the year. It must assign 51th week.

> January sales are significantly less than other months. This is the result of November and December high sales. After two high sales month, people prefer to pay less on January.

## Data Preprocessing and Cleaning

### Features
#### Extracted from train.csv
	•	Dept
	•	isHoliday

#### Extracted from stores.csv
	•	store
	•	type
	•	size

#### Extracted from Dates:
	•	time_index
	•	woy_sin
	•	woy_cos
	•	mon_sin
	•	mon_cos
	•	is_month_start
	•	is_month_end
	•	is_quarter_end
	•	is_year_end
	•	week_of_month
	•	dist_thanksgiving_wk
	•	dist_black_friday_wk
	•	dist_xmas_peak_wk
	•	dist_easter_wk
	•	dist_memorial_day_wk
	•	dist_july4_wk
	•	dist_labor_day_wk
	•	dist_super_bowl_wk
	•	is_black_friday_wk
	•	is_thanksgiving_wk
	•	is_xmas_peak_wk
	•	is_back_to_school

#### Label
	•	weekly_sales

## Date Features

In [38]:
def add_date_features(df, date_col="Date", clip_weeks=8):
    out = df.copy()
    d = pd.to_datetime(out[date_col]).dt.tz_localize(None).dt.normalize()

    # Core calendar
    out["time_index"] = ((d - d.min()).dt.days // 7).astype(int)

    woy = d.dt.isocalendar().week.astype(int)
    out["woy_sin"] = np.sin(2 * np.pi * woy / 52.1775)
    out["woy_cos"] = np.cos(2 * np.pi * woy / 52.1775)

    mon = d.dt.month
    out["mon_sin"] = np.sin(2 * np.pi * mon / 12)
    out["mon_cos"] = np.cos(2 * np.pi * mon / 12)

    out["is_month_start"] = d.dt.is_month_start.astype(int)
    out["is_month_end"] = d.dt.is_month_end.astype(int)
    out["is_quarter_end"] = d.dt.is_quarter_end.astype(int)
    out["is_year_end"] = d.dt.is_year_end.astype(int)

    out["week_of_month"] = (((d.dt.day - 1) // 7) + 1).clip(1, 5).astype(int)

    # US retail events & distances (weeks)
    from datetime import date, timedelta
    from calendar import monthrange

    def nth_weekday_of_month(y, m, weekday, n):
        first = date(y, m, 1)
        shift = (weekday - first.weekday()) % 7
        return first + timedelta(days=shift + 7 * (n - 1))

    def last_weekday_of_month(y, m, weekday):
        last_day = monthrange(y, m)[1]
        last = date(y, m, last_day)
        shift = (last.weekday() - weekday) % 7
        return last - timedelta(days=shift)

    def easter_sunday(y):
        a = y % 19
        b = y // 100;
        c = y % 100
        d0 = b // 4;
        e0 = b % 4
        f = (b + 8) // 25
        g = (b - f + 1) // 3
        h = (19 * a + b - d0 - g + 15) % 30
        i = c // 4;
        k = c % 4
        l = (32 + 2 * e0 + 2 * i - h - k) % 7
        m0 = (a + 11 * h + 22 * l) // 451
        month = (h + l - 7 * m0 + 114) // 31
        day = ((h + l - 7 * m0 + 114) % 31) + 1
        return date(y, month, day)

    def event_dates(y):
        tg = nth_weekday_of_month(y, 11, 3, 4)
        bf = tg + timedelta(days=1)
        xp = date(y, 12, 23)
        es = easter_sunday(y)
        md = last_weekday_of_month(y, 5, 0)
        j4 = date(y, 7, 4)
        ld = nth_weekday_of_month(y, 9, 0, 1)
        sb = nth_weekday_of_month(y, 2, 6, 1)
        return {"thanksgiving": tg, "black_friday": bf, "xmas_peak": xp,
                "easter": es, "memorial_day": md, "july4": j4,
                "labor_day": ld, "super_bowl": sb}

    years = d.dt.year.unique()
    year_to_events = {int(y): event_dates(int(y)) for y in years}

    def clipped_weeks(series, m):
        s = series.astype(int)
        return s.where(s.abs() <= m, 0)

    events = ["thanksgiving", "black_friday", "xmas_peak", "easter",
              "memorial_day", "july4", "labor_day", "super_bowl"]

    for ev in events:
        ev_dates = d.dt.year.map(lambda y: year_to_events[int(y)][ev])
        dist_w = (d.dt.date - ev_dates).map(lambda td: td.days // 7)
        out[f"dist_{ev}_wk"] = clipped_weeks(pd.Series(dist_w, index=out.index), clip_weeks).astype(int)

    out["is_black_friday_wk"] = (out["dist_black_friday_wk"] == 0).astype(int)
    out["is_thanksgiving_wk"] = (out["dist_thanksgiving_wk"] == 0).astype(int)
    out["is_xmas_peak_wk"] = (out["dist_xmas_peak_wk"] == 0).astype(int)

    y = d.dt.year.astype(str)
    bts_start = pd.to_datetime(y + "-07-15")
    bts_end = pd.to_datetime(y + "-09-10")
    out["is_back_to_school"] = ((d >= bts_start) & (d <= bts_end)).astype(int)

    return out

In [39]:
processed_data_df = add_date_features(raw_train_df)

## Store Features

In [40]:
processed_data_df = processed_data_df.merge(stores_df, on="Store", how="left")


## Re-order the Features

In [41]:
# Step 1: Sort rows by Date (chronologically)
processed_data_df = processed_data_df.sort_values("Date").reset_index(drop=True)

# Step 2: Reorder the columns as desired
desired_order = [
    "Date",
    "Store", "Type", "Size",  # Store-related
    "Dept", "IsHoliday",  # Base data
    "time_index",  # Trend
    "woy_sin", "woy_cos",  # Week of year (cyclic)
    "mon_sin", "mon_cos",  # Month (cyclic)
    "week_of_month",  # In-month position
    "is_month_start", "is_month_end", "is_quarter_end", "is_year_end",  # Calendar flags
    "dist_thanksgiving_wk", "dist_black_friday_wk", "dist_xmas_peak_wk",
    "dist_easter_wk", "dist_memorial_day_wk", "dist_july4_wk", "dist_labor_day_wk", "dist_super_bowl_wk",
    # Event distances
    "is_black_friday_wk", "is_thanksgiving_wk", "is_xmas_peak_wk", "is_back_to_school",  # Event flags
    "Weekly_Sales"  # Label
]

# Step 3: Reorder columns
processed_data_df = processed_data_df[desired_order]

# Preview
processed_data_df.head()

,Date,Store,Type,Size,Dept,IsHoliday,time_index,woy_sin,woy_cos,mon_sin,...,dist_easter_wk,dist_memorial_day_wk,dist_july4_wk,dist_labor_day_wk,dist_super_bowl_wk,is_black_friday_wk,is_thanksgiving_wk,is_xmas_peak_wk,is_back_to_school,Weekly_Sales
0,2010-02-05,1,A,151315,1,False,0,0.566372,0.82415,0.866025,...,0,0,0,0,-1,1,1,1,0,24924.50
1,2010-02-05,29,B,93638,5,False,0,0.566372,0.82415,0.866025,...,0,0,0,0,-1,1,1,1,0,15552.08
2,2010-02-05,29,B,93638,6,False,0,0.566372,0.82415,0.866025,...,0,0,0,0,-1,1,1,1,0,3200.22
3,2010-02-05,29,B,93638,7,False,0,0.566372,0.82415,0.866025,...,0,0,0,0,-1,1,1,1,0,10820.05
4,2010-02-05,29,B,93638,8,False,0,0.566372,0.82415,0.866025,...,0,0,0,0,-1,1,1,1,0,20055.64


In [42]:
# processed_data_df = processed_data_df.drop(columns=["Store"])
# processed_data_df = processed_data_df.drop(columns=["Date"])

## Save Processed Data

In [43]:
processed_data_path = processed_data_dir / 'processed_data.csv'
processed_data_df.to_csv(processed_data_path, index=False)
print(f"Processed data saved to '{'/'.join(processed_data_path.parts[-3:])}'.")

Processed data saved to 'data/processed/processed_data.csv'.


## Data Splitting

In [44]:
# Step 1: Safely parse mixed-format or inconsistent dates
processed_data_df["Date"] = pd.to_datetime(
    processed_data_df["Date"],
    dayfirst=True,
    format='mixed',
    errors='coerce'  # turn invalid dates into NaT (not crash)
)

# Step 2: Drop rows with unparseable (missing) dates
processed_data_df = processed_data_df.dropna(subset=["Date"])

# Step 3: Sort by Date (chronological order)
processed_data_df = processed_data_df.sort_values("Date").reset_index(drop=True)

# Step 4: Define split date for 80/20 time-aware split
split_date = pd.to_datetime("2012-04-13")

# Step 5: Split into train and validation
train_df = processed_data_df[processed_data_df["Date"] <= split_date]
valid_df = processed_data_df[processed_data_df["Date"] > split_date]

print(f"Train size: {len(train_df)} rows")
print(f"Valid size: {len(valid_df)} rows")


Train size: 338738 rows
Valid size: 82832 rows


## Dropping Meta Data

In [45]:
cols_to_drop = ["Date", "Store"]

train_df = train_df.drop(columns=cols_to_drop)
valid_df = valid_df.drop(columns=cols_to_drop)

In [50]:
train_df.to_csv(curated_data_dir / "train.csv", index=False)
print(f"Train.csv saved to '{'/'.join((curated_data_dir / "train.csv").parts[-3:])}'.")
valid_df.to_csv(curated_data_dir / "test.csv", index=False)
print(f"Test.csv saved to '{'/'.join((curated_data_dir / "test.csv").parts[-3:])}'.")

Train.csv saved to 'data/curated/train.csv'.
Test.csv saved to 'data/curated/test.csv'.


## Train.csv

In [47]:
train_df.head()

,Type,Size,Dept,IsHoliday,time_index,woy_sin,woy_cos,mon_sin,mon_cos,week_of_month,...,dist_easter_wk,dist_memorial_day_wk,dist_july4_wk,dist_labor_day_wk,dist_super_bowl_wk,is_black_friday_wk,is_thanksgiving_wk,is_xmas_peak_wk,is_back_to_school,Weekly_Sales
0,A,151315,1,False,0,0.566372,0.82415,0.866025,0.5,1,...,0,0,0,0,-1,1,1,1,0,24924.50
1,B,125833,97,False,0,0.566372,0.82415,0.866025,0.5,1,...,0,0,0,0,-1,1,1,1,0,668.48
2,B,125833,85,False,0,0.566372,0.82415,0.866025,0.5,1,...,0,0,0,0,-1,1,1,1,0,693.87
3,A,155078,80,False,0,0.566372,0.82415,0.866025,0.5,1,...,0,0,0,0,-1,1,1,1,0,8654.60
4,B,125833,55,False,0,0.566372,0.82415,0.866025,0.5,1,...,0,0,0,0,-1,1,1,1,0,11123.56


## Test.csv

In [48]:
valid_df.head()

,Type,Size,Dept,IsHoliday,time_index,woy_sin,woy_cos,mon_sin,mon_cos,week_of_month,...,dist_easter_wk,dist_memorial_day_wk,dist_july4_wk,dist_labor_day_wk,dist_super_bowl_wk,is_black_friday_wk,is_thanksgiving_wk,is_xmas_peak_wk,is_back_to_school,Weekly_Sales
338738,B,125833,17,False,115,0.937328,-0.348448,0.866025,-0.5,3,...,1,-6,0,0,0,1,1,1,0,4202.98
338739,A,184109,41,False,115,0.937328,-0.348448,0.866025,-0.5,3,...,1,-6,0,0,0,1,1,1,0,307.50
338740,A,203750,3,False,115,0.937328,-0.348448,0.866025,-0.5,3,...,1,-6,0,0,0,1,1,1,0,7245.43
338741,C,39910,23,False,115,0.937328,-0.348448,0.866025,-0.5,3,...,1,-6,0,0,0,1,1,1,0,23.01
338742,B,140167,4,False,115,0.937328,-0.348448,0.866025,-0.5,3,...,1,-6,0,0,0,1,1,1,0,19455.41
